In [328]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import utils as ut
import LIM_class
import torch
import torch.nn as nn
import torch.optim as optim
import json
import itertools
from torch.utils.data import DataLoader, Dataset

plt.style.use("../plotting.mplstyle")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [329]:
data = xr.open_dataset("./data/ts_Amon_CESM2_piControl_r1i1p1f1.nc")["ts"]
#data = xr.open_dataset("./data/zos_Amon_CESM2_piControl_r1i1p1f1.nc")["zos"]
#data = xr.open_dataset("./data/ssta_1950_2021.nc")["ssta"]

#14400 orginial size
data = data[:500, :, :]
print("Data shape: {}".format(data.shape))
print("Data : {}".format(data))


# Use Principal Component Analysis to reduce the dimensionality of the data

pca_10 = ut.SpatioTemporalPCA(data, n_components=20)
eof_10 = pca_10.eofs()
pc_10 = pca_10.principal_components()

Data shape: (500, 192, 288)
Data : <xarray.DataArray 'ts' (time: 500, lat: 192, lon: 288)>
[27648000 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 -180.0 -178.8 -177.5 -176.2 ... 176.2 177.5 178.8
  * time     (time) object 0001-01-15 12:00:00 ... 0042-08-15 12:00:00
Attributes: (12/19)
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        Surface temperature (skin for open ocean)
    description:    Surface temperature (skin for open ocean)
    frequency:      mon
    id:             ts
    ...             ...
    time_label:     time-mean
    time_title:     Temporal mean
    title:          Surface Temperature
    type:           real
    units:          K
    variable_id:    ts


In [330]:
# Create training and test data
data = pca_10.principal_components()
index_train = int(0.8 * len(data["time"]))
data_train = data[:, :index_train]
data_test = data[:, index_train:]

# Creating an example LIM object
tau = 1
model = LIM_class.LIM(tau)
model.fit(data_train.data)
#print("Data train : {} + shape: {} + type: {}".format(data_train.data[:5], data_train.data.shape, type(data_train.data)))

eigenvalues, _, _ = ut.matrix_decomposition(model.green_function)
t_decay = [abs(-(1/np.log(eigenvalue.real))) for eigenvalue in eigenvalues]

#print("Eigenvalues : min {} + max {}".format(min(eigenvalues), max(eigenvalues)))
#print("T-decay : min {} + max {}".format(min(t_decay), max(t_decay)))

Frobenius norm: 3.6571788787841797


C:\Users\felix\AppData\Local\Temp\ipykernel_1904\1943380292.py:14: RuntimeWarning: invalid value encountered in log
  t_decay = [abs(-(1/np.log(eigenvalue.real))) for eigenvalue in eigenvalues]


In [331]:
from FNN_model import *

# Set random seed for reproducibility
torch.manual_seed(42)

def hyperparameter_training_loop(dataloader):

    # Generate all possible combinations of hyperparameters
    combinations = list(itertools.product(*hyperparams.values()))

    overall_best_model = {"hidden_size": None,
                         "learning_rate": None,
                         "num_epochs": None,
                         "loss": float('inf')}

    # Iterate over each hyperparameter combination
    for i, params in enumerate(combinations):
        print(f"Testing parameter combination {i+1}/{len(combinations)}: {params}")

        hidden_size = params[0]
        learning_rate = params[1]
        num_epochs = params[2]

        model = FeedforwardNetwork(1, hidden_size)
        losses = train(model, dataloader, num_epochs, learning_rate)

        # Save the model and hyperparameters to a file
        result = {
            'hyperparameters': {
                'hidden_size': params[0],
                'learning_rate': params[1],
                'num_epochs': params[2],
                "losses": losses
            }
        }
        if losses[-1] < overall_best_model["loss"]:
            overall_best_model["hidden_size"] = params[0]
            overall_best_model["learning_rate"] = params[1]
            overall_best_model["num_epochs"] = params[2]
            overall_best_model["loss"] = losses[-1]
            overall_best_model["losses"] = losses

        filename = f"./model_trained/fnn_model_{i+1}.pt"
        with open(filename, 'w') as f:
            json.dump(result, f)

        print(f"Saved model and hyperparameters to {filename}\n")

    return overall_best_model


batch_size = 8

# Create the DataLoader for first principal component
data = np.array(data)[0]
#print("Data : {} + shape: {} + type: {}".format(data, data.shape, type(data)))
dataloader = create_dataloader(data, batch_size)

# Hyperparameter search space
hyperparams = {
    'hidden_size': [32],
    'learning_rate': [0.0005],
    'num_epochs': [1000, 2000, 3000]
}

#Train the model
#train(model, dataloader, num_epochs, learning_rate)
best_model = hyperparameter_training_loop(dataloader)

print(f"Best model: {best_model}")
plot_loss_evolution(best_model["losses"], np.arange(0,best_model["num_epochs"], 100, dtype=int), best_model["learning_rate"], best_model["hidden_size"])




Testing parameter combination 1/3: (32, 0.0005, 1000)
Epoch [1/1000], Loss: 173573568.12698412
Epoch [101/1000], Loss: 784959.4449404762
Epoch [201/1000], Loss: 785006.0124007936
Epoch [301/1000], Loss: 784955.689484127
Epoch [401/1000], Loss: 784905.3263888889
Epoch [501/1000], Loss: 784854.9280753968
Epoch [601/1000], Loss: 784804.621031746
Epoch [701/1000], Loss: 784754.2053571428
Epoch [801/1000], Loss: 784703.9737103175
Epoch [901/1000], Loss: 784653.7038690476
Saved model and hyperparameters to ./model_trained/fnn_model_1.pt

Testing parameter combination 2/3: (32, 0.0005, 2000)
Epoch [1/2000], Loss: 143017471.1111111
Epoch [101/2000], Loss: 785938.8030753968
Epoch [201/2000], Loss: 785969.5362103175
Epoch [301/2000], Loss: 785898.2698412698
Epoch [401/2000], Loss: 785822.6304563492
Epoch [501/2000], Loss: 785737.4295634921
Epoch [601/2000], Loss: 785674.5124007936
Epoch [701/2000], Loss: 785614.4692460317
Epoch [801/2000], Loss: 785554.4117063492
Epoch [901/2000], Loss: 785494.4

TypeError: only integer scalar arrays can be converted to a scalar index